In [1]:
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def read_images_txt(path):
    """
    从 images.txt 文件中读取相机位姿和图像名称。
    返回一个字典，键为 image_id，值为包含名称和位置的字典。
    """
    images = {}
    with open(path, "r") as f:
        lines = f.readlines()
        
    # 从第5行开始是实际数据 (跳过4行注释头)
    i = 4
    while i < len(lines):
        # 读取外参行
        line = lines[i]
        parts = line.split()
        
        image_id = int(parts[0])
        qw, qx, qy, qz = map(float, parts[1:5])
        tx, ty, tz = map(float, parts[5:8])
        camera_id = int(parts[8])
        name = parts[9]
        
        images[image_id] = {
            'name': name,
            'tvec': np.array([tx, ty, tz]),
            'qvec': np.array([qw, qx, qy, qz])
        }
        
        # 跳过下一行的 POINTS2D[] 数据
        i += 2
            
    return images

def main():
    """
    主函数：读取相机数据，划分训练/测试集，并生成交互式3D可视化图。
    """
    # --- 配置区 ---
    # 请确保此路径指向您正确的 images.txt 文件
    images_txt_path = "/root/autodl-tmp/gaussian-splatting/data/courtyard/sparse/0/images.txt"
    output_html_file = "camera_distribution.html"
    # --- 结束配置 ---

    if not os.path.exists(images_txt_path):
        print(f"❌ 错误: 文件未找到 at '{images_txt_path}'")
        return

    print(f"🔍 正在读取和分析: {images_txt_path}")

    # 1. 读取所有相机数据
    all_images = read_images_txt(images_txt_path)
    
    # 按 image_id 排序以确保划分逻辑与gaussian-splatting代码一致
    sorted_images = sorted(all_images.items(), key=lambda item: item[0])
    
    # 2. 准备用于可视化的数据
    camera_data = []
    for i, (image_id, data) in enumerate(sorted_images):
        set_type = "Test" if (i % 8 == 0) else "Train"
        camera_data.append({
            "id": image_id,
            "name": data['name'],
            "x": data['tvec'][0],
            "y": data['tvec'][1],
            "z": data['tvec'][2],
            "set": set_type
        })

    # 3. 将数据转换为Pandas DataFrame，方便绘图
    df = pd.DataFrame(camera_data)
    
    # 4. 计算训练集的几何中心，作为参考点
    train_df = df[df['set'] == 'Train']
    train_centroid = train_df[['x', 'y', 'z']].mean().values

    print("\n📊 正在生成交互式3D散点图...")

    # 5. 使用 Plotly 创建交互式3D图
    fig = px.scatter_3d(
        df,
        x='x', y='y', z='z',
        color='set',  # 根据 'set' 列（Train/Test）来区分颜色
        hover_name='name', # 鼠标悬停时显示文件名
        title="相机空间位置分布 (蓝色=训练集, 红色=测试集)",
        color_discrete_map={'Train': 'blue', 'Test': 'red'}, # 指定颜色
        symbol='set', # 使用不同符号
        size_max=10
    )

    # 6. 在图上单独添加训练集几何中心的标记
    fig.add_trace(go.Scatter3d(
        x=[train_centroid[0]],
        y=[train_centroid[1]],
        z=[train_centroid[2]],
        mode='markers',
        marker=dict(color='lime', size=10, symbol='cross'),
        name='训练集中心 (Centroid)'
    ))
    
    # 7. 更新图表布局和视角
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=40),
        legend_title_text='数据集划分'
    )

    # 8. 保存为HTML文件并提示用户
    fig.write_html(output_html_file)
    
    print("\n" + "="*80)
    print("✅ 可视化成功！")
    print(f"🔥 一个名为 '{output_html_file}' 的文件已在当前目录生成。")
    print("👉 请在您的浏览器中打开它，然后用鼠标拖动来旋转和查看3D分布。")
    print("="*80)
    print("\n观察指南:")
    print("  - 蓝色点 (●) 是训练集相机。")
    print("  - 红色点 (♦) 是测试集相机。")
    print("  - 亮绿色十字 (✚) 是所有训练相机的平均位置中心。")
    print("\n您会发现，红色点大部分都分布在蓝色点云团的外部边缘，远离中心。")


if __name__ == "__main__":
    # 确保依赖库已安装
    try:
        import pandas
        import plotly
    except ImportError:
        print("⚠️ 警告: 缺少必要的库。请先运行:")
        print("pip install pandas plotly")
    else:
        main()

🔍 正在读取和分析: /root/autodl-tmp/gaussian-splatting/data/courtyard/sparse/0/images.txt

📊 正在生成交互式3D散点图...

✅ 可视化成功！
🔥 一个名为 'camera_distribution.html' 的文件已在当前目录生成。
👉 请在您的浏览器中打开它，然后用鼠标拖动来旋转和查看3D分布。

观察指南:
  - 蓝色点 (●) 是训练集相机。
  - 红色点 (♦) 是测试集相机。
  - 亮绿色十字 (✚) 是所有训练相机的平均位置中心。

您会发现，红色点大部分都分布在蓝色点云团的外部边缘，远离中心。


In [3]:
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go

# 四元数转旋转矩阵的函数
def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[1] * qvec[3] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[1] * qvec[3] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]
    ])

def read_images_txt(path):
    """从 images.txt 读取相机的位置(tvec)和旋转(qvec)。"""
    images = {}
    with open(path, "r") as f:
        lines = f.readlines()
    i = 4
    while i < len(lines):
        line = lines[i]
        parts = line.split()
        image_id = int(parts[0])
        qvec = np.array(list(map(float, parts[1:5]))) # qw, qx, qy, qz
        tvec = np.array(list(map(float, parts[5:8]))) # tx, ty, tz
        name = parts[9]
        images[image_id] = {'name': name, 'tvec': tvec, 'qvec': qvec}
        i += 2
    return images

def main():
    """主函数：读取相机数据并生成带有朝向向量的3D交互式图。"""
    # --- 配置区 ---
    images_txt_path = "/root/autodl-tmp/gaussian-splatting/data/courtyard/sparse/0/images.txt"
    output_html_file = "camera_orientation.html"
    vector_length = 1.0  # 控制图中朝向向量的长度，以便于观察
    # --- 结束配置 ---

    if not os.path.exists(images_txt_path):
        print(f"❌ 错误: 文件未找到 at '{images_txt_path}'")
        return

    print(f"🔍 正在读取和分析: {images_txt_path}")

    all_images = read_images_txt(images_txt_path)
    sorted_images = sorted(all_images.items(), key=lambda item: item[0])

    camera_data = []
    for i, (image_id, data) in enumerate(sorted_images):
        set_type = "Test" if (i % 8 == 0) else "Train"
        camera_data.append({"id": image_id, "data": data, "set": set_type})

    # 计算训练集的几何中心
    train_positions = np.array([cam['data']['tvec'] for cam in camera_data if cam['set'] == 'Train'])
    train_centroid = np.mean(train_positions, axis=0)

    print("📊 正在生成带有朝向的交互式3D图...")

    # 创建一个空的图表对象
    fig = go.Figure()

    # --- 为训练集和测试集分别添加点和线 ---
    for set_type, color in [("Train", "blue"), ("Test", "red")]:
        subset = [cam for cam in camera_data if cam['set'] == set_type]
        if not subset: continue

        positions = np.array([cam['data']['tvec'] for cam in subset])
        names = [cam['data']['name'] for cam in subset]

        # 添加相机位置的散点
        fig.add_trace(go.Scatter3d(
            x=positions[:, 0], y=positions[:, 1], z=positions[:, 2],
            mode='markers',
            marker=dict(color=color, size=4),
            name=f'{set_type} Cameras',
            hovertext=names,
            hoverinfo='text'
        ))

        # 添加表示相机朝向的线段
        for cam in subset:
            tvec = cam['data']['tvec']
            qvec = cam['data']['qvec']
            
            # COLMAP 'qvec' to rotation matrix
            rot_mat = qvec2rotmat(qvec)
            
            # 在COLMAP中，tvec = -R^T * C, 所以相机中心 C = -R * tvec
            # 而我们通常直接使用tvec作为相机位置近似，这里假设tvec代表相机光学中心
            # 相机朝向是旋转矩阵的第三列（Z轴），乘以-1因为相机看向-Z方向
            direction_vector = -rot_mat[:, 2] 
            
            end_point = tvec + direction_vector * vector_length
            
            fig.add_trace(go.Scatter3d(
                x=[tvec[0], end_point[0]],
                y=[tvec[1], end_point[1]],
                z=[tvec[2], end_point[2]],
                mode='lines',
                line=dict(color=color, width=3),
                hoverinfo='none',
                showlegend=False # 每个小线段不单独显示图例
            ))

    # 添加训练集中心的标记
    fig.add_trace(go.Scatter3d(
        x=[train_centroid[0]], y=[train_centroid[1]], z=[train_centroid[2]],
        mode='markers',
        marker=dict(color='lime', size=10, symbol='cross'),
        name='训练集中心 (Centroid)'
    ))
    
    # 更新布局
    fig.update_layout(
        title="相机位置与朝向可视化 (线段表示朝向)",
        scene=dict(
            xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
            aspectratio=dict(x=1, y=1, z=1)
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig.write_html(output_html_file)

    print("\n" + "="*80)
    print(f"✅ 可视化成功！文件 '{output_html_file}' 已生成。")
    print("👉 请在浏览器中打开它，并仔细观察。")
    print("="*80)
    print("\n观察指南:")
    print("  - 每个点代表一个相机的位置。")
    print("  - 从每个点伸出的**短线**代表该相机的**朝向**。")
    print("  - **蓝色**代表训练集，**红色**代表测试集。")
    print("  - **亮绿色十字**是训练相机的几何中心。")
    print("\n**请重点检查**：红色的线段（测试集相机）是否都大致指向了绿色的十字？")
    print("如果大部分红色线段都**背离**了绿色十字，或者指向了奇怪的方向，那就找到了问题所在。")


if __name__ == "__main__":
    try:
        import pandas
        import plotly
    except ImportError:
        print("⚠️ 警告: 缺少必要的库。请先运行: pip install pandas plotly")
    else:
        main()

🔍 正在读取和分析: /root/autodl-tmp/gaussian-splatting/data/courtyard/sparse/0/images.txt
📊 正在生成带有朝向的交互式3D图...

✅ 可视化成功！文件 'camera_orientation.html' 已生成。
👉 请在浏览器中打开它，并仔细观察。

观察指南:
  - 每个点代表一个相机的位置。
  - 从每个点伸出的**短线**代表该相机的**朝向**。
  - **蓝色**代表训练集，**红色**代表测试集。
  - **亮绿色十字**是训练相机的几何中心。

**请重点检查**：红色的线段（测试集相机）是否都大致指向了绿色的十字？
如果大部分红色线段都**背离**了绿色十字，或者指向了奇怪的方向，那就找到了问题所在。


In [4]:
import os

def analyze_camera_ids(images_txt_path, cameras_txt_path):
    """
    读取 images.txt 和 cameras.txt，检查测试集图像关联的 CAMERA_ID 是否一致和有效。
    """
    # 1. 读取 cameras.txt，获取所有有效的 CAMERA_ID 及其信息
    valid_camera_ids = {}
    with open(cameras_txt_path, "r") as f:
        lines = f.readlines()
    for line in lines:
        if line.strip().startswith('#'):
            continue
        parts = line.split()
        cam_id = int(parts[0])
        model = parts[1]
        width, height = int(parts[2]), int(parts[3])
        params = " ".join(parts[4:])
        valid_camera_ids[cam_id] = f"Model: {model}, Resolution: {width}x{height}, Params: {params}"
    
    print("--- 发现的有效相机模型 (from cameras.txt) ---")
    if not valid_camera_ids:
        print("❌ 错误: 未能在 cameras.txt 中找到任何有效的相机模型！")
        return
    for cam_id, info in valid_camera_ids.items():
        print(f"  CAMERA_ID={cam_id} -> {info}")
    print("-" * 50)

    # 2. 读取 images.txt，检查每张图片的 CAMERA_ID
    print("\n--- 检查 images.txt 中的 CAMERA_ID 关联 ---")
    all_images_data = []
    with open(images_txt_path, "r") as f:
        lines = f.readlines()
    i = 4 # 跳过文件头
    while i < len(lines):
        line = lines[i]
        parts = line.split()
        image_id = int(parts[0])
        linked_camera_id = int(parts[8])
        name = parts[9]
        all_images_data.append({
            "name": name,
            "linked_camera_id": linked_camera_id
        })
        i += 2
        
    # 按默认顺序（即文件中的读取顺序）来确定测试集
    test_image_names = []
    issues_found = False
    
    print("\n[关键] 分析测试集图像的相机模型关联:")
    for i, image_data in enumerate(all_images_data):
        if i % 8 == 0:  # 默认的测试集划分规则
            test_image_names.append(image_data['name'])
            linked_id = image_data['linked_camera_id']
            print(f"  - 测试图像: {image_data['name']}")
            print(f"    - 关联的 CAMERA_ID: {linked_id}")
            
            # 检查这个ID是否在 cameras.txt 中定义过
            if linked_id not in valid_camera_ids:
                print("    - ❌ 严重错误: 这个 CAMERA_ID 在 cameras.txt 中不存在！")
                issues_found = True
            else:
                print(f"    - ✅ ID有效。关联的模型信息: {valid_camera_ids[linked_id]}")
            print("-" * 20)
            
    # 3. 检查训练集是否使用了不同的 CAMERA_ID
    train_camera_ids = set()
    for i, image_data in enumerate(all_images_data):
        if i % 8 != 0:
            train_camera_ids.add(image_data['linked_camera_id'])
            
    test_camera_ids = set()
    for i, image_data in enumerate(all_images_data):
        if i % 8 == 0:
            test_camera_ids.add(image_data['linked_camera_id'])

    print("\n--- 训练集 vs 测试集 CAMERA_ID 使用情况总结 ---")
    print(f"训练集使用的 CAMERA_ID 集合: {train_camera_ids}")
    print(f"测试集使用的 CAMERA_ID 集合: {test_camera_ids}")
    
    if len(test_camera_ids.difference(train_camera_ids)) > 0:
        print("⚠️ 警告: 测试集使用了一些训练集中从未出现过的 CAMERA_ID！")
        issues_found = True
        
    if len(train_camera_ids) > 1 or len(test_camera_ids) > 1:
        print("ℹ️ 信息: 数据集内使用了多种不同的相机模型。")

    print("\n--- 最终诊断 ---")
    if issues_found:
        print("✅ 问题找到！测试集图像关联到了无效或与训练集不一致的相机内参模型(CAMERA_ID)。")
        print("这会导致渲染时使用错误的焦距或分辨率，从而产生模糊/错误的图像。")
        print("请检查您的数据预处理流程，确保所有图像都正确地关联到了 cameras.txt 中定义的相机模型。")
    else:
        print("🤔 诊断未确定。测试集和训练集使用了相同且有效的相机内参模型。")
        print("如果问题依然存在，可能的原因包括：")
        print("  1. 测试集的原始图像文件(.JPG)本身损坏或分辨率极低。")
        print("  2. 3DGS代码中存在一个更深层次的、与数据加载相关的未知Bug。")


def main():
    # --- 配置区 ---
    base_path = "/root/autodl-tmp/gaussian-splatting/data/courtyard/sparse/0/"
    images_txt_path = os.path.join(base_path, "images.txt")
    cameras_txt_path = os.path.join(base_path, "cameras.txt")
    # --- 结束配置 ---

    if not os.path.exists(images_txt_path) or not os.path.exists(cameras_txt_path):
        print("❌ 错误: 确保 images.txt 和 cameras.txt 文件都在指定路径下。")
        return
        
    analyze_camera_ids(images_txt_path, cameras_txt_path)

if __name__ == "__main__":
    main()

--- 发现的有效相机模型 (from cameras.txt) ---
  CAMERA_ID=3 -> Model: PINHOLE, Resolution: 6208x4134, Params: 3408.35 3408.8 3114.7 2070.92
  CAMERA_ID=2 -> Model: PINHOLE, Resolution: 6200x4134, Params: 3407.41 3408.08 3112.83 2065.6
  CAMERA_ID=1 -> Model: PINHOLE, Resolution: 6205x4135, Params: 3409.58 3409.44 3115.16 2064.73
  CAMERA_ID=0 -> Model: PINHOLE, Resolution: 6198x4129, Params: 3411.42 3410.02 3116.72 2062.52
--------------------------------------------------

--- 检查 images.txt 中的 CAMERA_ID 关联 ---

[关键] 分析测试集图像的相机模型关联:
  - 测试图像: dslr_images_undistorted/DSC_0323.JPG
    - 关联的 CAMERA_ID: 0
    - ✅ ID有效。关联的模型信息: Model: PINHOLE, Resolution: 6198x4129, Params: 3411.42 3410.02 3116.72 2062.52
--------------------
  - 测试图像: dslr_images_undistorted/DSC_0315.JPG
    - 关联的 CAMERA_ID: 1
    - ✅ ID有效。关联的模型信息: Model: PINHOLE, Resolution: 6205x4135, Params: 3409.58 3409.44 3115.16 2064.73
--------------------
  - 测试图像: dslr_images_undistorted/DSC_0298.JPG
    - 关联的 CAMERA_ID: 3
    - ✅ ID有效。关联的

In [5]:
import os

# --- 配置区 ---
images_txt_path = "/root/autodl-tmp/gaussian-splatting/data/courtyard/sparse/0/images.txt"
# --- 结束配置 ---

def fix_problematic_test_image(file_path):
    """
    通过调换位置，将使用'CAMERA_ID=0'的测试图片从测试集中排除。
    """
    print(f"🔍 正在读取文件: {file_path}")
    with open(file_path, "r") as f:
        lines = f.readlines()

    # 文件头是4行注释
    header = lines[:4]
    image_data_lines = lines[4:]
    
    # 将图像数据两行一组进行处理
    images = []
    for i in range(0, len(image_data_lines), 2):
        images.append(image_data_lines[i:i+2])
        
    target_cam_id_to_find = 0
    problematic_image_index = -1

    # 找到那张被选为测试集且使用了错误ID的图片
    # 默认规则是 i % 8 == 0
    for i in range(len(images)):
        if i % 8 == 0:
            line_parts = images[i][0].split()
            linked_camera_id = int(line_parts[8])
            
            if linked_camera_id == target_cam_id_to_find:
                problematic_image_index = i
                image_name = line_parts[9]
                print(f"✅ 找到问题图片!")
                print(f"   - 索引位置: {i}")
                print(f"   - 文件名: {image_name}")
                print(f"   - 使用的 CAMERA_ID: {linked_camera_id}")
                break
    
    if problematic_image_index == -1:
        print("ℹ️ 未在测试集中找到使用 CAMERA_ID=0 的图片。文件可能已经修复或无需修改。")
        return

    # 确保我们不会换到文件末尾之外
    if problematic_image_index + 1 >= len(images):
        print("❌ 错误: 问题图片是最后一张，无法进行交换。请手动处理。")
        return
        
    # 与下一张图片交换位置
    print(f"\n🔄 正在将索引 {problematic_image_index} 的图片与索引 {problematic_image_index + 1} 的图片进行交换...")
    
    temp = images[problematic_image_index]
    images[problematic_image_index] = images[problematic_image_index + 1]
    images[problematic_image_index + 1] = temp

    # 将交换后的数据写回文件
    new_content = "".join(header)
    for image_pair in images:
        new_content += "".join(image_pair)
        
    with open(file_path, "w") as f:
        f.write(new_content)
        
    print("\n" + "="*50)
    print("🎉 文件修复成功！")
    print("现在，有问题的图片位于训练集位置，而一张正常的图片换到了测试集位置。")
    print("您可以直接重新运行评估脚本了。")
    print("="*50)


if __name__ == "__main__":
    if not os.path.exists(images_txt_path):
        print(f"❌ 错误: 文件未找到 at '{images_txt_path}'")
    else:
        # 在操作前再次提醒备份
        if not os.path.exists(images_txt_path + ".original"):
             print("⚠️ 警告: 未检测到备份文件 (.original)。强烈建议先手动备份！")
        fix_problematic_test_image(images_txt_path)

🔍 正在读取文件: /root/autodl-tmp/gaussian-splatting/data/courtyard/sparse/0/images.txt
✅ 找到问题图片!
   - 索引位置: 0
   - 文件名: dslr_images_undistorted/DSC_0323.JPG
   - 使用的 CAMERA_ID: 0

🔄 正在将索引 0 的图片与索引 1 的图片进行交换...

🎉 文件修复成功！
现在，有问题的图片位于训练集位置，而一张正常的图片换到了测试集位置。
您可以直接重新运行评估脚本了。


In [6]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ==============================================================================
#                      HELPER FUNCTIONS (FROM PREVIOUS SCRIPTS)
# ==============================================================================

def qvec2rotmat(qvec):
    """四元数转旋转矩阵"""
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2, 2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3], 2 * qvec[1] * qvec[3] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3], 1 - 2 * qvec[1]**2 - 2 * qvec[3]**2, 2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[1] * qvec[3] - 2 * qvec[0] * qvec[2], 2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1], 1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]
    ])

def read_cameras_txt(path):
    """读取 cameras.txt 文件，返回一个包含相机模型信息的字典。"""
    valid_camera_ids = {}
    with open(path, "r") as f:
        lines = f.readlines()
    for line in lines:
        if line.strip().startswith('#'): continue
        parts = line.split()
        cam_id = int(parts[0])
        model, width, height = parts[1], int(parts[2]), int(parts[3])
        params = " ".join(parts[4:])
        valid_camera_ids[cam_id] = f"Model: {model}, Resolution: {width}x{height}, Params: {params}"
    return valid_camera_ids

def read_images_txt(path):
    """读取 images.txt 文件，返回一个包含所有图像详细信息的列表。"""
    all_images_data = []
    with open(path, "r") as f:
        lines = f.readlines()
    i = 4 # Skip header
    while i < len(lines):
        line = lines[i]
        parts = line.split()
        all_images_data.append({
            "image_id": int(parts[0]),
            "qvec": np.array(list(map(float, parts[1:5]))),
            "tvec": np.array(list(map(float, parts[5:8]))),
            "camera_id": int(parts[8]),
            "name": parts[9]
        })
        i += 2
    return all_images_data

# ==============================================================================
#                      ANALYSIS AND VISUALIZATION FUNCTIONS
# ==============================================================================

def perform_text_analysis(camera_data, valid_camera_models):
    """在终端打印详细的文本分析报告。"""
    print("\n" + "="*80)
    print("PART 1: DETAILED TEXT ANALYSIS REPORT")
    print("="*80)

    # 按默认规则划分
    test_indices = {i for i in range(len(camera_data)) if i % 8 == 0}
    
    train_positions = np.array([cam['tvec'] for i, cam in enumerate(camera_data) if i not in test_indices])
    train_centroid = np.mean(train_positions, axis=0)
    
    print(f"训练相机几何中心 (平均位置): {np.round(train_centroid, 2)}")
    print("\n--- [关键] 逐一分析被选为测试集的相机 ---")

    issues_found = False
    test_cam_ids = set()

    for i in sorted(list(test_indices)):
        cam = camera_data[i]
        distance = np.linalg.norm(cam['tvec'] - train_centroid)
        linked_id = cam['camera_id']
        test_cam_ids.add(linked_id)
        
        print(f"\n  - 测试图像: {cam['name']} (文件中的第 {i+1} 张)")
        print(f"    - 位置: {np.round(cam['tvec'], 2)}, 与训练中心距离: {distance:.2f} 米")
        print(f"    - 关联的 CAMERA_ID: {linked_id}")
        
        if linked_id not in valid_camera_models:
            print("    - ❌ 严重错误: 这个 CAMERA_ID 在 cameras.txt 中不存在！")
            issues_found = True
        else:
            print(f"    - ✅ ID有效。关联模型: {valid_camera_models[linked_id]}")

    train_cam_ids = {cam['camera_id'] for i, cam in enumerate(camera_data) if i not in test_indices}

    print("\n--- 训练集 vs 测试集 CAMERA_ID 使用情况总结 ---")
    print(f"训练集使用的 CAMERA_ID 集合: {train_cam_ids}")
    print(f"测试集使用的 CAMERA_ID 集合: {test_cam_ids}")

    unseen_ids = test_cam_ids.difference(train_cam_ids)
    if unseen_ids:
        print(f"    - ⚠️ 警告: 测试集使用了训练集中从未出现过的 CAMERA_ID: {unseen_ids}")
        issues_found = True
        
    return issues_found, unseen_ids

def create_3d_visualization(camera_data, output_html_file):
    """生成包含位置和朝向的3D交互式可视化文件。"""
    print("\n" + "="*80)
    print("PART 2: GENERATING 3D INTERACTIVE VISUALIZATION")
    print("="*80)

    fig = go.Figure()
    vector_length = 0.5  # 可视化向量的长度

    test_indices = {i for i in range(len(camera_data)) if i % 8 == 0}
    train_centroid = np.mean(np.array([c['tvec'] for i, c in enumerate(camera_data) if i not in test_indices]), axis=0)

    # 分别处理训练集和测试集
    for set_type, color, indices in [("Train", "blue", {i for i in range(len(camera_data)) if i not in test_indices}), 
                                     ("Test", "red", test_indices)]:
        subset = [camera_data[i] for i in sorted(list(indices))]
        if not subset: continue

        positions = np.array([cam['tvec'] for cam in subset])
        names = [cam['name'] for cam in subset]

        fig.add_trace(go.Scatter3d(x=positions[:, 0], y=positions[:, 1], z=positions[:, 2], mode='markers',
                                   marker=dict(color=color, size=4), name=f'{set_type} Cameras',
                                   hovertext=names, hoverinfo='text'))

        for cam in subset:
            tvec, qvec = cam['tvec'], cam['qvec']
            rot_mat = qvec2rotmat(qvec)
            direction_vector = -rot_mat[:, 2]
            end_point = tvec + direction_vector * vector_length
            fig.add_trace(go.Scatter3d(x=[tvec[0], end_point[0]], y=[tvec[1], end_point[1]], z=[tvec[2], end_point[2]],
                                       mode='lines', line=dict(color=color, width=3),
                                       hoverinfo='none', showlegend=False))

    fig.add_trace(go.Scatter3d(x=[train_centroid[0]], y=[train_centroid[1]], z=[train_centroid[2]], mode='markers',
                               marker=dict(color='lime', size=10, symbol='cross'), name='训练集中心'))
    
    fig.update_layout(title="相机位置与朝向可视化 (蓝色=训练, 红色=测试)",
                      scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectratio=dict(x=1, y=1, z=1)),
                      margin=dict(l=0, r=0, b=0, t=40))

    fig.write_html(output_html_file)
    print(f"✅ 可视化成功！文件 '{output_html_file}' 已生成。")
    print("👉 请在浏览器中打开它，并仔细观察红色点/线的分布和朝向。")

# ==============================================================================
#                                MAIN EXECUTION
# ==============================================================================

def main():
    """主函数，执行对指定场景的全面诊断。"""
    # --- [ 配置区 ] ---
    # 只需修改 SCENE_NAME 即可对不同场景进行分析
    SCENE_NAME = "pipes"
    # --- [ 结束配置 ] ---
    
    # 自动构建路径
    base_path = f"/root/autodl-tmp/gaussian-splatting/data/{SCENE_NAME}/sparse/0/"
    images_txt_path = os.path.join(base_path, "images.txt")
    cameras_txt_path = os.path.join(base_path, "cameras.txt")
    output_html_file = f"{SCENE_NAME}_camera_analysis.html"

    print(f"🚀🚀🚀 开始对场景 '{SCENE_NAME}' 进行全面诊断 🚀🚀🚀")

    # 检查文件是否存在
    if not all(os.path.exists(p) for p in [images_txt_path, cameras_txt_path]):
        print(f"❌ 错误: 找不到必要的COLMAP文件。请确保以下路径正确:\n - {images_txt_path}\n - {cameras_txt_path}")
        return

    # 1. 读取数据
    valid_camera_models = read_cameras_txt(cameras_txt_path)
    all_camera_data = read_images_txt(images_txt_path)
    if not valid_camera_models or not all_camera_data:
        print("❌ 错误: 读取相机或图像数据失败，无法继续分析。")
        return

    # 2. 执行文本分析
    issues_found, unseen_ids = perform_text_analysis(all_camera_data, valid_camera_models)
    
    # 3. 生成3D可视化
    create_3d_visualization(all_camera_data, output_html_file)

    # 4. 输出最终诊断结论
    print("\n" + "="*80)
    print("FINAL DIAGNOSIS")
    print("="*80)
    if issues_found:
        print(f"✅ 问题确认！诊断结果指向【相机内参不匹配】。")
        print(f"   - 原因: 测试集中的部分图像使用了相机模型ID {unseen_ids}，而这个模型从未在训练集中出现过。")
        print(f"   - 影响: 这会导致渲染器使用未经优化的、不兼容的相机参数，从而产生错误的、模糊的图像（'浓雾'现象）。")
        print("\n   - 解决方案建议:")
        print("     1. (推荐) 修改 `images.txt` 文件，将所有图像的 CAMERA_ID 统一为训练集中最常用的一个。")
        print("     2. 或者，通过调换 `images.txt` 中的行顺序，将使用这些 '未见过' ID的图像从测试集中排除。")
    else:
        print("✅ 诊断完成。根据文件分析，该场景的测试集划分看起来是【健康的】。")
        print("   - 相机位置和朝向分布合理。")
        print("   - 测试集使用的相机内参模型也都在训练集中出现过。")
        print("\n   - 如果您在该场景上仍然遇到恒定的低分问题，可能的原因包括：")
        print("     1. 该场景本身的重建难度极高，模型过拟合严重。")
        print("     2. 原始的测试集.JPG图像文件本身存在问题（如损坏、全黑等）。")
        print("     3. 3DGS代码中存在针对此类场景的特定Bug。")
    print("="*80)


if __name__ == "__main__":
    try:
        import pandas
        import plotly
    except ImportError:
        print("⚠️ 警告: 缺少必要的库。请先运行: pip install pandas plotly")
    else:
        main()

🚀🚀🚀 开始对场景 'pipes' 进行全面诊断 🚀🚀🚀

PART 1: DETAILED TEXT ANALYSIS REPORT
训练相机几何中心 (平均位置): [-0.23  0.21  0.29]

--- [关键] 逐一分析被选为测试集的相机 ---

  - 测试图像: dslr_images_undistorted/DSC_0647.JPG (文件中的第 1 张)
    - 位置: [-0.17  0.24  1.28], 与训练中心距离: 0.99 米
    - 关联的 CAMERA_ID: 0
    - ✅ ID有效。关联模型: Model: PINHOLE, Resolution: 6220x4141, Params: 3430.27 3429.23 3119.2 2057.75

  - 测试图像: dslr_images_undistorted/DSC_0639.JPG (文件中的第 9 张)
    - 位置: [0.72 0.19 0.59], 与训练中心距离: 0.99 米
    - 关联的 CAMERA_ID: 0
    - ✅ ID有效。关联模型: Model: PINHOLE, Resolution: 6220x4141, Params: 3430.27 3429.23 3119.2 2057.75

--- 训练集 vs 测试集 CAMERA_ID 使用情况总结 ---
训练集使用的 CAMERA_ID 集合: {0}
测试集使用的 CAMERA_ID 集合: {0}

PART 2: GENERATING 3D INTERACTIVE VISUALIZATION
✅ 可视化成功！文件 'pipes_camera_analysis.html' 已生成。
👉 请在浏览器中打开它，并仔细观察红色点/线的分布和朝向。

FINAL DIAGNOSIS
✅ 诊断完成。根据文件分析，该场景的测试集划分看起来是【健康的】。
   - 相机位置和朝向分布合理。
   - 测试集使用的相机内参模型也都在训练集中出现过。

   - 如果您在该场景上仍然遇到恒定的低分问题，可能的原因包括：
     1. 该场景本身的重建难度极高，模型过拟合严重。
     2. 原始的测试集.JPG图像文件本身存在问题（如损坏、全黑等）。
   